### MAST30034: Applied Data Science Project 1
---
# Preprocessing Part 1: Cleaning The Data
#### Xavier Travers (1178369)

Cleaning the datasets of null, inconsistent, or unnecessary values.

In [1]:
# imports used throughout this notebook
from collections import defaultdict
import sys
from pyspark.sql import functions as F
import pandas as pd

# add homemade helpers
sys.path.insert(1, '../../scripts')
import helpers.cleaning_helpers as ch
import helpers.join_helpers as jh

# path where the data files are stored
DATA_PATH = '../../data'

In [2]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName('MAST30034 XT Project 1')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.sql.repl.eagerEval.enabled', True) 
    .config('spark.sql.parquet.cacheMetadata', 'true')
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/21 08:24:34 WARN Utils: Your hostname, Polaris resolves to a loopback address: 127.0.1.1; using 172.18.201.145 instead (on interface eth0)
22/08/21 08:24:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/21 08:24:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/21 08:24:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# import the cdc week file to convert all dates to cdc weeks now
mmwr_weeks_df = spark.read.parquet(f'{DATA_PATH}/raw/virals/mmwr_weeks.parquet')
mmwr_weeks_df.limit(5)

year,month,day,month_mmwr_index,week_index,us_format,week_ending,week_month,week_year,borough,__index_level_0__
2019,12,29,1,1,12/29/2019,2020-01-04,1,2020,Bronx,0
2019,12,30,1,1,12/30/2019,2020-01-04,1,2020,Bronx,1
2019,12,31,1,1,12/31/2019,2020-01-04,1,2020,Bronx,2
2020,1,1,1,1,01/01/2020,2020-01-04,1,2020,Bronx,3
2020,1,2,1,1,01/02/2020,2020-01-04,1,2020,Bronx,4


In [4]:
# import the zones dataset
zones_df = spark.read.csv(f'{DATA_PATH}/raw/tlc_zones/zones.csv',
    header = True)
zones_df.limit(5)

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham G...,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


### 1. Join population statistics with zones

In [ ]:
# import the populations data
pop_df_2010_2019 = pd.read_excel(f'{DATA_PATH}/raw/populations/2010_2019.xlsx',
    header = [3], index_col=0).transpose()

pop_df_2010_2019.head(5)

,New York,".Albany County, New York",".Allegany County, New York",".Bronx County, New York",".Broome County, New York",".Cattaraugus County, New York",".Cayuga County, New York",".Chautauqua County, New York",".Chemung County, New York",".Chenango County, New York",...,".Washington County, New York",".Wayne County, New York",".Westchester County, New York",".Wyoming County, New York",".Yates County, New York","Note: The estimates are based on the 2010 Census and reflect changes to the April 1, 2010 population due to the Count Question Resolution program and geographic program revisions. All geographic boundaries for the 2019 population estimates are as of January 1, 2019. For population estimates methodology statements, see http://www.census.gov/programs-surveys/popest/technical-documentation/methodology.html.",Suggested Citation:,"Annual Estimates of the Resident Population for Counties in New York: April 1, 2010 to July 1, 2019 (CO-EST2019-ANNRES-36)","Source: U.S. Census Bureau, Population Division",Release Date: March 2020
Census,19378102.0,304204.0,48946.0,1385108.0,200600.0,80317.0,80026.0,134905.0,88830.0,50477.0,...,63216.0,93772.0,949113.0,42155.0,25348.0,NaN,NaN,NaN,NaN,NaN
Estimates Base,19378144.0,304208.0,48923.0,1384580.0,200675.0,80337.0,80008.0,134907.0,88847.0,50511.0,...,63254.0,93751.0,949218.0,42154.0,25364.0,NaN,NaN,NaN,NaN,NaN
2010,19399878.0,304086.0,48971.0,1387298.0,200481.0,80218.0,79895.0,134725.0,88895.0,50399.0,...,63356.0,93751.0,950601.0,42126.0,25376.0,NaN,NaN,NaN,NaN,NaN
2011,19499241.0,304596.0,48800.0,1397335.0,199363.0,79815.0,79693.0,134209.0,88899.0,50182.0,...,63091.0,93256.0,956262.0,41849.0,25454.0,NaN,NaN,NaN,NaN,NaN
2012,19572932.0,305723.0,48210.0,1411496.0,198667.0,79348.0,79505.0,133304.0,89137.0,49883.0,...,63003.0,93029.0,959585.0,41700.0,25337.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
# extract the new york city boroughs
pop_df_2010_2019 = pop_df_2010_2019[[
    '.Bronx County, New York',
    '.Kings County, New York',
    '.New York County, New York',
    '.Queens County, New York',
    '.Richmond County, New York']]

# rename the columns to their corresponding boroughs
pop_df_2010_2019.columns = [
    'Bronx',
    'Brooklyn',
    'Manhattan',
    'Queens',
    'Staten Island'
]
pop_df_2010_2019.head(5)

,Bronx,Brooklyn,Manhattan,Queens,Staten Island
Census,1385108.0,2504700.0,1585873.0,2230722.0,468730.0
Estimates Base,1384580.0,2504721.0,1586381.0,2230619.0,468730.0
2010,1387298.0,2509828.0,1588767.0,2234701.0,469615.0
2011,1397335.0,2540817.0,1608293.0,2255482.0,471021.0
2012,1411496.0,2568450.0,1623911.0,2272222.0,470614.0


In [ ]:
# import the populations data
pop_df_2020_2021 = pd.read_excel(f'{DATA_PATH}/raw/populations/2020_2021.xlsx',
    header = [3], index_col=0).transpose()

pop_df_2020_2021.head(5)

,New York,".Albany County, New York",".Allegany County, New York",".Bronx County, New York",".Broome County, New York",".Cattaraugus County, New York",".Cayuga County, New York",".Chautauqua County, New York",".Chemung County, New York",".Chenango County, New York",...,".Washington County, New York",".Wayne County, New York",".Westchester County, New York",".Wyoming County, New York",".Yates County, New York","Note: The estimates are developed from a base that incorporates the 2020 Census, Vintage 2020 estimates, and 2020 Demographic Analysis estimates. For population estimates methodology statements, see http://www.census.gov/programs-surveys/popest/technical-documentation/methodology.html. The estimates feature geographic boundaries from the Vintage 2020 estimates series; the geographic boundaries for these 2021 population estimates are as of January 1, 2020.",Suggested Citation:,"Annual Estimates of the Resident Population for Counties in New York: April 1, 2020 to July 1, 2021\n(CO-EST2021-POP-36)","Source: U.S. Census Bureau, Population Division",Release Date: March 2022
Unnamed: 1,20201249.0,314848.0,46456.0,1472654.0,198683.0,77042.0,76248.0,127657.0,84148.0,47220.0,...,61302.0,91283.0,1004457.0,40531.0,24774.0,NaN,NaN,NaN,NaN,NaN
2020,20154933.0,314368.0,46373.0,1466438.0,198199.0,76907.0,76095.0,127424.0,83882.0,47073.0,...,61143.0,91103.0,1003245.0,40401.0,24709.0,NaN,NaN,NaN,NaN,NaN
2021,19835913.0,313743.0,46106.0,1424948.0,197240.0,76426.0,75880.0,126807.0,83045.0,46537.0,...,60956.0,90923.0,997895.0,40491.0,24613.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
# extract the new york city boroughs
pop_df_2020_2021 = pop_df_2020_2021[[
    '.Bronx County, New York',
    '.Kings County, New York',
    '.New York County, New York',
    '.Queens County, New York',
    '.Richmond County, New York']]

# rename the columns to their corresponding boroughs
pop_df_2020_2021.columns = [
    'Bronx',
    'Brooklyn',
    'Manhattan',
    'Queens',
    'Staten Island'
]
pop_df_2020_2021.head(5)

,Bronx,Brooklyn,Manhattan,Queens,Staten Island
Unnamed: 1,1472654.0,2736074.0,1694251.0,2405464.0,495747.0
2020,1466438.0,2727393.0,1687834.0,2395791.0,495522.0
2021,1424948.0,2641052.0,1576876.0,2331143.0,493494.0


In [ ]:
# stack the population data
pop_df = pd.concat(
    [pop_df_2010_2019, pop_df_2020_2021]
)

# only keep 2018-2021 data
pop_df = pop_df.filter(items = [2018, 2019, 2020, 2021], axis = 0)
pop_df.head()

,Bronx,Brooklyn,Manhattan,Queens,Staten Island
2018,1432087.0,2578074.0,1629055.0,2274605.0,476260.0
2019,1418207.0,2559903.0,1628706.0,2253858.0,476143.0
2020,1466438.0,2727393.0,1687834.0,2395791.0,495522.0
2021,1424948.0,2641052.0,1576876.0,2331143.0,493494.0


In [ ]:
# rename the index column
pop_df.index.name = 'week_year'
pop_df = pop_df.reset_index()
pop_df.head()

,week_year,Bronx,Brooklyn,Manhattan,Queens,Staten Island
0,2018,1432087.0,2578074.0,1629055.0,2274605.0,476260.0
1,2019,1418207.0,2559903.0,1628706.0,2253858.0,476143.0
2,2020,1466438.0,2727393.0,1687834.0,2395791.0,495522.0
3,2021,1424948.0,2641052.0,1576876.0,2331143.0,493494.0


In [ ]:
# verticalize the data (I just want a year column and a borough column)
temp_df = None
for col in pop_df.columns[1:]:
    # add the borough to an extracted borough column
    # using .copy() hides a space-hog warning about not editing pop_df 
    # (which is my intention)
    b_pop_df = pop_df[['week_year', col]].copy()
    b_pop_df.columns = ['week_year', 'population']
    b_pop_df['borough'] = col

    if temp_df is None:
        temp_df = b_pop_df 
    else:
        temp_df = pd.concat([temp_df, b_pop_df])

# set the verticalized data to the population data
pop_df = spark.createDataFrame(temp_df)
pop_df.limit(5)

week_year,population,borough
2018,1432087.0,Bronx
2019,1418207.0,Bronx
2020,1466438.0,Bronx
2021,1424948.0,Bronx
2018,2578074.0,Brooklyn


In [ ]:
# save the population df
pop_df.write.mode('overwrite').parquet(f'{DATA_PATH}/curated/population_by_borough_by_year')

### 2. Cleaning the TLC dataset(s)

In [ ]:
# read in an example to see the datatypes
example_df = spark.read.parquet(f'{DATA_PATH}/raw/tlc/yellow/2019_12.parquet/')
example_df.limit(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2019-12-01 00:26:58,2019-12-01 00:41:45,1.0,4.2,1.0,N,142,116,2,14.5,3.0,0.5,0.0,0.0,0.3,18.3,2.5,null
1,2019-12-01 00:12:08,2019-12-01 00:12:14,1.0,0.0,1.0,N,145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0,null
1,2019-12-01 00:25:53,2019-12-01 00:26:04,1.0,0.0,1.0,N,145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0,null
1,2019-12-01 00:12:03,2019-12-01 00:33:19,2.0,9.4,1.0,N,138,25,1,28.5,0.5,0.5,10.0,0.0,0.3,39.8,0.0,null
1,2019-12-01 00:05:27,2019-12-01 00:16:32,2.0,1.6,1.0,N,161,237,2,9.0,3.0,0.5,0.0,0.0,0.3,12.8,2.5,null


In [ ]:
# there are large trip distances that I need to filter out
example_df.sort('trip_distance', ascending = False).limit(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
2,2019-12-04 10:20:13,2019-12-04 10:24:43,1.0,19130.18,5.0,N,224,224,2,11.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,null
2,2019-12-25 20:44:07,2019-12-26 03:10:41,2.0,363.13,5.0,N,132,265,2,400.0,0.0,0.5,0.0,46.34,0.3,447.14,0.0,null
2,2019-12-09 21:17:33,2019-12-10 02:55:18,4.0,323.81,5.0,N,130,265,2,400.0,0.0,0.0,0.0,126.0,0.3,526.3,0.0,null
1,2019-12-29 12:29:25,2019-12-29 12:34:03,1.0,300.8,1.0,N,231,125,2,5.5,2.5,0.5,0.0,0.0,0.3,8.8,2.5,null
1,2019-12-15 10:44:46,2019-12-15 14:45:38,1.0,248.6,5.0,N,138,265,1,475.0,0.0,0.0,8.88,18.62,0.3,502.8,0.0,null


In [ ]:
# read in the tlc data
tlc_df = jh.read_stacked_tlc_df(spark)
tlc_df.limit(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2019-12-01 00:26:58,2019-12-01 00:41:45,1.0,4.2,1.0,N,142,116,2,14.5,3.0,0.5,0.0,0.0,0.3,18.3,2.5,null
1,2019-12-01 00:12:08,2019-12-01 00:12:14,1.0,0.0,1.0,N,145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0,null
1,2019-12-01 00:25:53,2019-12-01 00:26:04,1.0,0.0,1.0,N,145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0,null
1,2019-12-01 00:12:03,2019-12-01 00:33:19,2.0,9.4,1.0,N,138,25,1,28.5,0.5,0.5,10.0,0.0,0.3,39.8,0.0,null
1,2019-12-01 00:05:27,2019-12-01 00:16:32,2.0,1.6,1.0,N,161,237,2,9.0,3.0,0.5,0.0,0.0,0.3,12.8,2.5,null


In [ ]:
# check the amount of raw rows
tlc_df.count()

64913648

In [ ]:
# derive extra values which are used to filter out valid trips
SECONDS_TO_HOURS = 1 / (60*60)
tlc_df = tlc_df\
    .withColumn('hours_elapsed', 
        ( # counts how long the trip was in hours
            (F.col("tpep_dropoff_datetime").cast("long")
            - F.col('tpep_pickup_datetime').cast("long")) 
            * SECONDS_TO_HOURS
        )
    )\
    .withColumn('mph', 
        ( # calculate the speed of the trip
            F.col('trip_distance') / F.col('hours_elapsed')
        )
    )

In [ ]:
# check that the columns look correct
tlc_df.limit(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,hours_elapsed,mph
1,2019-12-01 00:26:58,2019-12-01 00:41:45,1.0,4.2,1.0,N,142,116,2,14.5,3.0,0.5,0.0,0.0,0.3,18.3,2.5,null,0.24638888888888888,17.04622322435175
1,2019-12-01 00:12:08,2019-12-01 00:12:14,1.0,0.0,1.0,N,145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0,null,0.001666666666666...,0.0
1,2019-12-01 00:25:53,2019-12-01 00:26:04,1.0,0.0,1.0,N,145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0,null,0.003055555555555...,0.0
1,2019-12-01 00:12:03,2019-12-01 00:33:19,2.0,9.4,1.0,N,138,25,1,28.5,0.5,0.5,10.0,0.0,0.3,39.8,0.0,null,0.35444444444444445,26.52037617554859
1,2019-12-01 00:05:27,2019-12-01 00:16:32,2.0,1.6,1.0,N,161,237,2,9.0,3.0,0.5,0.0,0.0,0.3,12.8,2.5,null,0.18472222222222223,8.661654135338345


In [ ]:
# https://ypdcrime.com/vt/article30.php?zoom_highlight=fifty+five+miles+per+hour#t1180-a
# As per: https://www.dot.ny.gov/divisions/operating/oom/transportation-systems/repository/TSMI-17-05.pdf
# the NYS maximum speed limit is 65 mph. filter out trips faster than legal.
tlc_df = tlc_df.where(
    (F.col('mph').isNotNull()) &
    (F.col('mph') <= 65)
)

# this legitimately removes any invalid trips 
# (and other null values are culled later)

In [ ]:
# WARNING: this one is time intensive 
# check for large distance trips 
# (as long as the timing seems reasonable, they are kept)
tlc_df.sort('trip_distance', ascending = False).limit(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,hours_elapsed,mph
2,2021-10-12 21:32:01,2021-10-13 04:57:45,1.0,448.47,5.0,N,216,265,2,400.0,0.0,0.0,0.0,6.55,0.3,406.85,0.0,0.0,7.428888888888889,60.368381693090036
1,2020-06-02 06:36:15,2020-06-02 14:33:50,1.0,441.6,5.0,N,68,265,2,300.0,0.0,0.0,0.0,0.0,0.3,300.3,0.0,null,7.959722222222222,55.479322980282674
1,2021-01-20 11:22:05,2021-01-20 19:47:56,1.0,427.7,1.0,Y,4,265,1,1128.5,2.5,0.5,1140.44,20.16,0.3,2292.4,2.5,null,8.430833333333334,50.73045369180586
1,2020-07-30 15:10:02,2020-07-30 22:04:34,1.0,414.4,5.0,N,138,265,1,400.0,0.0,0.0,87.2,35.74,0.3,523.24,0.0,null,6.908888888888889,59.98070119009328
2,2020-12-06 07:33:27,2020-12-06 14:01:29,1.0,407.78,5.0,N,264,265,1,200.0,0.0,0.0,1.0,11.75,0.3,215.55,2.5,null,6.467222222222222,63.05334593248002


In [ ]:
# WARNING: this one is time intensive 
# next, filter out trips which do not start  within the 5 boroughs 
tlc_df = ch.extract_borough_name(
    tlc_df.withColumnRenamed('PULocationID', 'pu_location_id'), zones_df,  'pu')
# tlc_df = ch.extract_borough_name(tlc_df, zones_df,  'do')
tlc_df.sort('trip_distance', ascending=False).limit(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,pu_location_id,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,hours_elapsed,mph,pu_borough
2,2021-10-12 21:32:01,2021-10-13 04:57:45,1.0,448.47,5.0,N,216,265,2,400.0,0.0,0.0,0.0,6.55,0.3,406.85,0.0,0.0,7.428888888888889,60.368381693090036,Queens
1,2020-06-02 06:36:15,2020-06-02 14:33:50,1.0,441.6,5.0,N,68,265,2,300.0,0.0,0.0,0.0,0.0,0.3,300.3,0.0,null,7.959722222222222,55.479322980282674,Manhattan
1,2021-01-20 11:22:05,2021-01-20 19:47:56,1.0,427.7,1.0,Y,4,265,1,1128.5,2.5,0.5,1140.44,20.16,0.3,2292.4,2.5,null,8.430833333333334,50.73045369180586,Manhattan
1,2020-07-30 15:10:02,2020-07-30 22:04:34,1.0,414.4,5.0,N,138,265,1,400.0,0.0,0.0,87.2,35.74,0.3,523.24,0.0,null,6.908888888888889,59.98070119009328,Queens
2,2020-02-08 08:58:50,2020-02-08 15:03:36,3.0,369.94,3.0,N,48,265,2,960.5,0.0,0.0,0.0,0.0,0.3,960.8,0.0,null,6.079444444444444,60.85095494836882,Manhattan


In [ ]:
# names of the tlc datasets to clean 
# (I was originally planning on working on fhvhv and green as well)
TLC_NAMES = ['yellow']

# dictionary to rename all the columns I want to keep
TLC_KEEP_COLUMNS = {
    'tpep_pickup_datetime': 'date',
    # 'passenger_count': 'passengers',
    'trip_distance': 'trip_distance',
    'pu_borough': 'pu_borough',
    # 'DOLocationID': 'do_location_id',
    'hours_elapsed': 'hours_elapsed'
    # #  below only apply to fhvhv
    # 'hvfhs_license_num': 'fhvhv_license',
    # 'pickup_datetime': 'date',
    # 'trip_miles': 'trip_distance',
    # 'shared_request_flag': 'shared'
}

# create a dictionary of the columns to keep and the required filters
TLC_CLEAN_COLUMNS = {
    'pu_location_id': [ch.non_null], 
    # 'do_location_id': [ch.non_null], 
    # 'passengers': [ch.non_null], 
    'trip_distance': [ch.non_null, ch.non_negative], 
    # #  below only apply to fhvhv
    # 'fhvhv_license': [ch.non_null], 
}

In [ ]:
# perform the drawn out cleaning process (function in `scripts/helpers`)
tlc_df = ch.perform_cleaning(tlc_df, mmwr_weeks_df, TLC_KEEP_COLUMNS, 
    TLC_CLEAN_COLUMNS)

In [ ]:
# WARNING: this one is time intensive 
# make sure that the latest data pads the used timeline 
# (so I'm not missing any weeks in the final timeline)
tlc_df.sort('week_index', ascending = False).limit(5)

year,month,day,week_ending,week_year,week_month,week_index,date,trip_distance,pu_borough,hours_elapsed
2022,1,1,2022-01-01,2021,12,105,01/01/2022,8.46,Queens,0.3136111111111111
2022,1,1,2022-01-01,2021,12,105,01/01/2022,8.65,Queens,0.2544444444444444
2022,1,1,2022-01-01,2021,12,105,01/01/2022,2.44,Queens,0.12055555555555555
2022,1,1,2022-01-01,2021,12,105,01/01/2022,13.03,Queens,0.35388888888888886
2022,1,1,2022-01-01,2021,12,105,01/01/2022,10.01,Queens,0.4211111111111111


In [ ]:
# WARNING: this one is time intensive 
# double check for any trip distance outliers
tlc_df.sort('trip_distance', ascending=False).limit(5)

# my cleaning seems to have worked

year,month,day,week_ending,week_year,week_month,week_index,date,trip_distance,pu_borough,hours_elapsed
2021,10,12,2021-10-16,2021,10,94,10/12/2021,448.47,Queens,7.428888888888889
2020,6,2,2020-06-06,2020,6,23,06/02/2020,441.6,Manhattan,7.959722222222222
2021,1,20,2021-01-23,2021,1,56,01/20/2021,427.7,Manhattan,8.430833333333334
2020,7,30,2020-08-01,2020,7,31,07/30/2020,414.4,Queens,6.908888888888889
2020,2,8,2020-02-08,2020,2,6,02/08/2020,369.94,Manhattan,6.079444444444444


In [ ]:
# WARNING: this one is time intensive 
# count the number of rows after cleaning
tlc_df.count()

55439045

In [ ]:
# WARNING: this one is time intensive 
# save the stacked df by month (this will take a while)
# sorting first prevents the partitioning part of the write from crashing
# (form personal experience)
tlc_df = tlc_df.sort('week_year', 'week_month')
tlc_df.write\
    .partitionBy('week_year', 'week_month')\
    .mode('overwrite')\
    .parquet(f'{DATA_PATH}/curated/tlc/cleaned/yellow')

### 3. Cleaning the COVID dataset

In [ ]:
# read in the covid dataset
covid_df = spark.read.csv(f'{DATA_PATH}/raw/virals/covid/cases_by_day.csv',
    header = True, inferSchema=True)
covid_df.limit(5)

22/08/21 08:16:11 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_PROBABLE_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_PROBABLE_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_PROBABLE_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_PROBABLE_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_PROBABLE_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
02/29/2020,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
03/01/2020,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
03/02/2020,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
03/03/2020,1,0,7,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
03/04/2020,5,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# sum the number of incomplete datasets (ensure no incomplete values)
sum(covid_df.select('INCOMPLETE'))

Column<'(INCOMPLETE + 0)'>

In [ ]:
# dictionary to rename all the columns I want to keep
COVID_KEEP_COLUMNS = {
    'date_of_interest':'date'
}

# create a dictionary of the columns to keep and the required filters
COVID_CLEAN_COLUMNS = defaultdict(lambda: ch.non_negative)

# define the boroughs as they appear in columns of the covid dataset
COVID_BOROUGHS = {
    'BX_':'Bronx',
    'BK_':'Brooklyn',
    'MN_':'Manhattan',
    'QN_':'Queens',
    'SI_':'Staten Island',
}

# define the count values to extract
COVID_COUNTS = {
    'CASE_COUNT': 'cases', 
    # 'DEATH_COUNT': 'deaths', 
    # 'HOSPITALIZED_COUNT': 'hospitalised'
}

# got throuch each borough and count value and add them to the columns to keep
for prefix, new_prefix in COVID_BOROUGHS.items():
    for suffix, new_suffix in COVID_COUNTS.items():
        COVID_KEEP_COLUMNS[f'{prefix}{suffix}'] = f'{new_prefix}{new_suffix}'

In [ ]:
# perform the drawn out cleaning process (function in `scripts/helpers`)
covid_df = ch.perform_cleaning(covid_df, mmwr_weeks_df, COVID_KEEP_COLUMNS, 
    COVID_CLEAN_COLUMNS)

In [ ]:
# date columns to keep after verticalising the covid data
COVID_DATE_COLUMNS = [
    F.col('date'), 
    F.col('week_ending'), 
    F.col('week_year'), 
    F.col('week_month'), 
    F.col('week_index')
]

# verticalise this dataset
# I'd rather just have a 'borough' column for homogeneity of all the data
temp_df = None
for prefix in COVID_BOROUGHS.values():
    # derive the columns for this borough to extract and stack
    borough_columns = []
    for suffix in COVID_COUNTS.values():
        borough_columns.append(F.col(f'{prefix}{suffix}').alias(suffix))

    # extract the counts for this borough and add them to the stacked dataframe
    if temp_df == None:
        temp_df = covid_df.select(COVID_DATE_COLUMNS + borough_columns)\
            .withColumn('borough', F.lit(prefix))
    else:
        temp_df = temp_df\
            .union(
                covid_df.select(COVID_DATE_COLUMNS + borough_columns)\
                    .withColumn('borough', F.lit(prefix))
            )

# set the df to the stacked data
covid_df = temp_df

In [ ]:
# fill the null cases (created from the one-sided outer join for the df) with 0
covid_df = covid_df.fillna(0, 'cases')

In [ ]:
# double check that dates of weeks are correctly added/derived
covid_df.sort('week_index', 'date').limit(5)

date,week_ending,week_year,week_month,week_index,cases,borough
null,2020-01-04,2020,1,1,0,Bronx
null,2020-01-04,2020,1,1,0,Bronx
null,2020-01-04,2020,1,1,0,Bronx
null,2020-01-04,2020,1,1,0,Bronx
null,2020-01-04,2020,1,1,0,Bronx


In [ ]:
# save the cleaned covid data
covid_df.write.mode('overwrite').parquet(f'{DATA_PATH}/curated/virals/covid/cleaned/cases_by_day')

### 4. Cleaning the flu dataset

In [9]:
# read in the flu dataset
flu_df = spark.read.csv(f'{DATA_PATH}/raw/virals/flu/cases_by_week.csv',
    header=True, inferSchema=True)
flu_df.limit(5)

Season,Region,County,CDC Week,Week Ending Date,Disease,Count,County Centroid,FIPS
2012-2013,NYC,RICHMOND,10,03/09/2013,INFLUENZA_A,0,"(40.5795, -74.1502)",36085
2011-2012,CAPITAL DISTRICT,ALBANY,10,03/10/2012,INFLUENZA_UNSPECI...,0,"(42.5882713, -73....",36001
2009-2010,CAPITAL DISTRICT,SCHENECTADY,41,10/17/2009,INFLUENZA_UNSPECI...,0,"(42.8175421, -74....",36093
2010-2011,WESTERN,CHAUTAUQUA,19,05/14/2011,INFLUENZA_B,0,"(42.3042159, -79....",36013
2013-2014,METRO,DUTCHESS,44,11/02/2013,INFLUENZA_A,0,"(41.7550085, -73....",36027


In [10]:
# get the list of distinct counties (column now called 'borough')
flu_df.select('County').distinct().limit(5)

County
FULTON
CATTARAUGUS
STEUBEN
YATES
KINGS


In [11]:
# map the boroughs to their proper names
# from: https://portal.311.nyc.gov/article/?kanumber=KA-02877
# also from map dict
FLU_COUNTY_TO_BOROUGH = {
    'BRONX': 'Bronx',
    'KINGS': 'Brooklyn',
    'NEW YORK': 'Manhattan',
    'QUEENS': 'Queens',
    'RICHMOND': 'Staten Island'
}

In [12]:
# apply the mapping to the flu df
flu_df = ch.replace_column_using_dict(flu_df, 'County', FLU_COUNTY_TO_BOROUGH)

In [13]:
# dictionary to rename all the columns I want to keep
FLU_KEEP_COLUMNS = {
    'Week Ending Date': 'date',
    'County': 'borough',
    'Disease': 'disease',
    'Count': 'cases',
}

# create a dictionary of the columns to keep and the required filters
FLU_CLEAN_COLUMNS = {
    'date': [],
    'region': [lambda _: F.col('region') == 'NYC'],
    'borough': [],
    'disease': [],
    'cases': [ch.non_negative]
}

In [14]:
# perform the drawn out cleaning process (function in `scripts/helpers`)
flu_df = ch.perform_cleaning(flu_df, mmwr_weeks_df, FLU_KEEP_COLUMNS, 
    FLU_CLEAN_COLUMNS)

In [15]:
# fill the null cases (created from the one-sided outer join for the df) with 0
flu_df = flu_df.fillna(0, 'cases')

In [16]:
# check that converting the boroughs worked
flu_df.limit(5)

year,month,day,week_ending,week_year,week_month,week_index,date,borough,disease,cases
2019,12,29,2020-01-04,2020,1,1,null,Bronx,null,0
2019,12,30,2020-01-04,2020,1,1,null,Bronx,null,0
2019,12,31,2020-01-04,2020,1,1,null,Bronx,null,0
2020,1,1,2020-01-04,2020,1,1,null,Bronx,null,0
2020,1,2,2020-01-04,2020,1,1,null,Bronx,null,0


In [17]:
# save the cleaned flu data
flu_df.write.mode('overwrite').parquet(f'{DATA_PATH}/curated/virals/flu/cleaned/cases_by_week')